In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('./')
sys.path.append('../')

In [4]:
import zarr
from candidate_graph import create_candidate_graph
import daisy
import linajea
from copy import deepcopy

In [7]:
candidate_graph = create_candidate_graph('anno_alice_T2030_tiny_2.zarr')

/
 ├── .zarr
 ├── Fragment_stats
 │   ├── Position
 │   │   ├── 0 (38, 4) int64
 │   │   ├── 1 (34, 4) int64
 │   │   └── 2 (42, 4) int64
 │   ├── Volume
 │   │   ├── 0 (38,) float64
 │   │   ├── 1 (34,) float64
 │   │   └── 2 (42,) float64
 │   └── id
 │       ├── 0 (38,) int64
 │       ├── 1 (34,) int64
 │       └── 2 (42,) int64
 ├── Fragments (3, 15, 30, 30) uint64
 ├── Merge_tree
 │   ├── Merge
 │   │   ├── 0 (37, 3) uint64
 │   │   ├── 1 (33, 3) uint64
 │   │   └── 2 (41, 3) uint64
 │   └── Scoring
 │       ├── 0 (37,) float64
 │       ├── 1 (33,) float64
 │       └── 2 (41,) float64
 ├── Raw (4, 3, 15, 30, 30) float64
 └── gt_trackimage (3, 15, 30, 30) uint64
frame: 0 / 2
-merged pre
-merged next
-added nodes pre
-added nodes next
-iterating over A&B
--iterating A: 0 / 75
--iterating A: 1 / 75
--iterating A: 2 / 75
--iterating A: 3 / 75
--iterating A: 4 / 75
--iterating A: 5 / 75
--iterating A: 6 / 75
--iterating A: 7 / 75
--iterating A: 8 / 75
--iterating A: 9 / 75
--iterating 

In [85]:
G = candidate_graph

In [86]:
G.nodes(data=True)

NodeDataView({5735095648387072: {'t': 0, 'z': 0, 'y': 23, 'x': 5, 'score': 0.5124858021736145, 'size': 163, 'parent': 6368414345986048, 'id': 5735095648387072}, 809283545464832: {'t': 0, 'z': 0, 'y': 18, 'x': 5, 'score': 0.0001, 'size': 23, 'parent': 5735095648387072, 'id': 809283545464832}, 4925855092441088: {'t': 0, 'z': 0, 'y': 24, 'x': 5, 'score': 0.0001, 'size': 140, 'parent': 5735095648387072, 'id': 4925855092441088}, 11540697387632128: {'t': 0, 'z': 3, 'y': 2, 'x': 26, 'score': 0.5264488458633423, 'size': 328, 'parent': 75646597586749440, 'id': 11540697387632128}, 2427936428785664: {'t': 0, 'z': 0, 'y': 3, 'x': 25, 'score': 0.0001, 'size': 69, 'parent': 11540697387632128, 'id': 2427936428785664}, 9112975713503232: {'t': 0, 'z': 4, 'y': 2, 'x': 26, 'score': 0.0001, 'size': 259, 'parent': 11540697387632128, 'id': 9112975713503232}, 6368414345986048: {'t': 0, 'z': 0, 'y': 23, 'x': 5, 'score': 0.5339860916137695, 'size': 181, 'parent': 7846157973716992, 'id': 6368414345986048}, 6333

In [87]:
import networkx as nx

nx.write_graphml(candidate_graph, 'candidate_graph.graphml')

In [88]:
from linajea_cost_test import get_merge_graph ,get_merge_graph_from_array
t = 0
z = zarr.open('anno_alice_T2030_tiny_2.zarr','r') 
merge_tree = z['Merge_tree/Merge/'+str(t)]
scores = z['Merge_tree/Scoring/'+str(t)]
merge_tree = get_merge_graph_from_array(merge_tree,scores)

g = get_merge_graph(G,0)

In [89]:
# check the root nodes 
import networkx as nx
for node in candidate_graph.nodes:
    if  candidate_graph.nodes[node]['parent'] == node:
        print(candidate_graph.nodes[node])


{'t': 0, 'z': 7, 'y': 14, 'x': 14, 'score': 0.9081573486328125, 'size': 13500, 'parent': 474989143487680000, 'id': 474989143487680000}
{'t': 1, 'z': 7, 'y': 14, 'x': 14, 'score': 0.8916463255882263, 'size': 13500, 'parent': 474989143487680001, 'id': 474989143487680001}
{'t': 2, 'z': 7, 'y': 14, 'x': 14, 'score': 0.9649198055267334, 'size': 13500, 'parent': 474989143487680002, 'id': 474989143487680002}


In [90]:
roi = daisy.Roi((0, 0, 0, 0), (3, 15, 25, 25))

    # input graph
graph = linajea.tracking.TrackGraph(G, frame_key='t', roi=roi)

In [91]:
from linajea_cost_test import get_edge_indicator_costs, get_node_indicator_costs, get_constraints


import time
import logging

logger = logging.getLogger(__name__)


class SolvingConfig():
    def __init__(self, solve_config):
        self.solve = solve_config

In [92]:
ps = {
            "track_cost": 4.0,
            "weight_edge_score": -0.1,
            "weight_node_score": -0.1,
            "selection_constant": -1.0,
            "max_cell_move": 0.0,
            "block_size": [5, 100, 100, 100],
            }
job = {"num_workers": 5, "queue": "normal"}

solve_config = linajea.config.SolveConfig(parameters=ps,
                                            job=job,
                                            context=[2, 100, 100, 100],
                                            solver_type = None,
                                            timeout = 100000)


In [93]:
config = SolvingConfig(solve_config)
constrs = get_constraints()

linajea.tracking.track(
                graph,
                config,
                frame_key='t',
                selected_key='selected',
                edge_indicator_costs=get_edge_indicator_costs,
                node_indicator_costs=get_node_indicator_costs,
                constraints_fns=constrs[0],
                pin_constraints_fns=constrs[1]
                )


Could not create Gurobi backend: Gurobi error in pylp/impl/solvers/GurobiBackend.cpp:22: No Gurobi license found (user daniel, host Daniels-Air.home, hostid fa63c4e5, cores 8)
feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       235 del vars, 471 del conss, 0 add conss, 118 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 222 clqs
(round 2, fast)       262 del vars, 471 del conss, 0 add conss, 118 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 222 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver (3 rounds): 1 aggregations, 0 fixings, 0 bound changes
(round 3, exhaustive) 263 del vars, 478 del conss, 0 add conss, 118 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 223 clqs
(round 4, exhaustive) 263 del vars, 478 del conss, 0 add conss, 118 chg bounds, 0 chg sides, 0 chg coeffs, 3993 upgd conss, 0 impls, 223 clqs
(round 5, medium)     401 del vars, 4040 del co

In [94]:
nx.write_graphml(graph,'test_solver.graphml')

set the parameter from the tomal file. 
- TODO for grid search

In [75]:
from linajea.config import TrackingConfig
config_file = 'config.toml'
config = TrackingConfig.from_file(config_file)

config.solve.parameters

[SolveParametersConfig(track_cost=4, weight_node_score=-0.1, selection_constant=-1, weight_division=1, division_constant=2.5, weight_child=2.0, weight_continuation=-1.0, weight_edge_score=-0.1, cell_state_key=None, block_size=[3, 15, 25, 25], max_cell_move=0, roi=None, feature_func='noop', val=True, tag=None),
 SolveParametersConfig(track_cost=3, weight_node_score=-1, selection_constant=-0.5, weight_division=5, division_constant=2.5, weight_child=1.0, weight_continuation=-1.0, weight_edge_score=-1, cell_state_key=None, block_size=[3, 15, 25, 25], max_cell_move=45, roi=None, feature_func='noop', val=True, tag=None),
 SolveParametersConfig(track_cost=1, weight_node_score=-1, selection_constant=2, weight_division=-10, division_constant=2.5, weight_child=2.0, weight_continuation=-1.0, weight_edge_score=-1, cell_state_key=None, block_size=[3, 15, 25, 25], max_cell_move=45, roi=None, feature_func='noop', val=True, tag=None),
 SolveParametersConfig(track_cost=7, weight_node_score=-13, selecti

In [76]:


roi = daisy.Roi((0, 0, 0, 0), (3, 15, 25, 25))
graph = linajea.tracking.TrackGraph(G,frame_key='t',roi=roi)
constrs = get_constraints()

# get sovlver paremeters 
parameters = deepcopy(config.solve.parameters)


selected_keys = ['selected_' + str(i) for i in range(len(parameters))]


job = {'num_workers':24,'queue': 'normal'}

solve_config = linajea.config.SolveConfig(parameters=parameters,
                                            job=job,
                                            context = [10,100,100,100],
                                            solver_type = None,
                                            timeout = 10000000)

config = SolvingConfig(solve_config)
linajea.tracking.track(
            graph,
            config,
            frame_key='t',
            selected_key=selected_keys,
            edge_indicator_costs=get_edge_indicator_costs,
            node_indicator_costs=get_node_indicator_costs,
            constraints_fns=constrs[0],
            pin_constraints_fns=constrs[1]
            )

Could not create Gurobi backend: Gurobi error in pylp/impl/solvers/GurobiBackend.cpp:22: No Gurobi license found (user daniel, host Daniels-Air.home, hostid fa63c4e5, cores 8)
feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       160 del vars, 326 del conss, 0 add conss, 83 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 262 clqs
   (0.0s) running MILP presolver
   (0.1s) MILP presolver (2 rounds): 1 aggregations, 0 fixings, 0 bound changes
(round 2, exhaustive) 161 del vars, 327 del conss, 0 add conss, 83 chg bounds, 0 chg sides, 0 chg coeffs, 5717 upgd conss, 0 impls, 268 clqs
(round 3, medium)     311 del vars, 5501 del conss, 10342 add conss, 83 chg bounds, 148 chg sides, 149 chg coeffs, 5717 upgd conss, 0 impls, 10604 clqs
(round 4, exhaustive) 311 del vars, 10672 del conss, 10342 add conss, 83 chg bounds, 148 chg sides, 290 chg coeffs, 5717 upgd conss, 0 impls, 10604 clqs
(round 5, medium)     

In [77]:

selected_edges = []
for u,v,data in graph.edges(data=True):
    if data['selected_1']:
        selected_edges.append((u,v))

In [78]:
candidate_graph.nodes(data=True)

NodeDataView({5735095648387072: {'t': 0, 'z': 0, 'y': 23, 'x': 5, 'score': 0.5124858021736145, 'size': 163, 'parent': 6368414345986048, 'id': 5735095648387072}, 809283545464832: {'t': 0, 'z': 0, 'y': 18, 'x': 5, 'score': 0.0001, 'size': 23, 'parent': 5735095648387072, 'id': 809283545464832}, 4925855092441088: {'t': 0, 'z': 0, 'y': 24, 'x': 5, 'score': 0.0001, 'size': 140, 'parent': 5735095648387072, 'id': 4925855092441088}, 11540697387632128: {'t': 0, 'z': 3, 'y': 2, 'x': 26, 'score': 0.5264488458633423, 'size': 328, 'parent': 75646597586749440, 'id': 11540697387632128}, 2427936428785664: {'t': 0, 'z': 0, 'y': 3, 'x': 25, 'score': 0.0001, 'size': 69, 'parent': 11540697387632128, 'id': 2427936428785664}, 9112975713503232: {'t': 0, 'z': 4, 'y': 2, 'x': 26, 'score': 0.0001, 'size': 259, 'parent': 11540697387632128, 'id': 9112975713503232}, 6368414345986048: {'t': 0, 'z': 0, 'y': 23, 'x': 5, 'score': 0.5339860916137695, 'size': 181, 'parent': 7846157973716992, 'id': 6368414345986048}, 6333

In [79]:
nx.write_graphml(graph,'solver1.graphml')